In [132]:
using DataFrames, Random, CSV, Statistics, DataStructures, Missings

In [133]:
ENV["LINES"] = 20

20

In [134]:
df = DataFrame(CSV.File("../data/2_Lifetech_DA_test (scoring).csv"))
first(df, 5)

,ID,MAX_DEBT,target (60+),IS_BILLING,RATINGCLASS,NEW_MSISDN,MEDIAN_LT,CNT_SMART_SALES
,Int64,Int64,Int64,Int64,String?,Int64,String?,Int64?
1,1,0,0,1,missing,1,"13,64",1
2,2,4,0,0,D3,1,missing,missing
3,3,2,0,0,-10006,1,missing,missing
4,4,4,0,1,B2,0,"44,98",0
5,5,157,1,1,-10006,1,"17,34",1


In [135]:
names(df)

15-element Vector{String}:
 "ID"
 "MAX_DEBT"
 "target (60+)"
 "IS_BILLING"
 "RATINGCLASS"
 "NEW_MSISDN"
 "MEDIAN_LT"
 "CNT_SMART_SALES"
 "REV_CNT_MNTH"
 "MAX_DAYS_W_OUTGOING"
 "MAX_DAYS_W_MB"
 "MAX_DAYS_IN_DEBT"
 "MAX_DEBT_SUM"
 "Column14"
 "Column15"

In [147]:
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,ID,8605.5,1,8605.5,17210,0,Int64
2,MAX_DEBT,51.4468,0,6.0,571,0,Int64
3,target (60+),0.153806,0,0.0,1,0,Int64
4,IS_BILLING,0.65491,0,1.0,1,0,Int64
5,RATINGCLASS,,-10006,,Unknown,0,String
6,NEW_MSISDN,0.75369,0,1.0,1,0,Int64
7,MEDIAN_LT,24.5102,0.0,24.5102,119.26,0,Float64
8,CNT_SMART_SALES,0.891381,0,1.0,7,4459,"Union{Missing, Int64}"
9,REV_CNT_MNTH,4.68088,0,6.0,6,4425,"Union{Missing, Int64}"


In [137]:
counter(df."RATINGCLASS")

Accumulator{Union{Missing, String}, Int64} with 19 entries:
  "-10006" => 3659
  "C3"     => 791
  "B2"     => 1854
  "2"      => 1300
  "B1"     => 1428
  "B3"     => 1153
  "C1"     => 1057
  "C2"     => 958
  "A2"     => 119
  "E1"     => 254
  "D2"     => 491
  "E3"     => 146
  "E2"     => 136
  "A3"     => 571
  "D3"     => 350
  ⋮        => ⋮

In [138]:
df."RATINGCLASS" = coalesce.(df."RATINGCLASS", "Unknown")

17210-element Vector{String}:
 "Unknown"
 "D3"
 "-10006"
 "B2"
 "-10006"
 "2"
 "B1"
 "E1"
 ⋮
 "F"
 "Unknown"
 "E1"
 "C2"
 "2"
 "-10006"
 "B2"

In [139]:
df.MEDIAN_LT = passmissing(replace).(df.MEDIAN_LT, ","=>".")
df.MEDIAN_LT = passmissing(parse).(Float64, df.MEDIAN_LT)
median_MEDIAN_LT = median(skipmissing(df.MEDIAN_LT))
df.MEDIAN_LT = coalesce.(df.MEDIAN_LT, median_MEDIAN_LT)

17210-element Vector{Float64}:
 13.64
 24.51021473814307
 24.51021473814307
 44.98
 17.34
 19.86
 27.0
 24.51021473814307
  ⋮
 17.69
 11.64
 31.08
 55.43
 37.49
 10.05
 11.59

In [140]:
counter(df.CNT_SMART_SALES)

Accumulator{Union{Missing, Int64}, Int64} with 9 entries:
  0       => 5325
  4       => 158
  5       => 36
  7       => 2
  2       => 2020
  6       => 6
  missing => 4459
  3       => 630
  1       => 4574

In [141]:
counter(df.REV_CNT_MNTH)

Accumulator{Union{Missing, Int64}, Int64} with 8 entries:
  5       => 1360
  4       => 715
  6       => 7836
  0       => 1258
  2       => 497
  missing => 4425
  3       => 528
  1       => 591

In [142]:
counter(df.MAX_DAYS_W_OUTGOING)

Accumulator{Union{Missing, Int64}, Int64} with 186 entries:
  56  => 33
  35  => 36
  60  => 28
  67  => 42
  73  => 47
  115 => 49
  112 => 50
  86  => 42
  168 => 140
  183 => 83
  177 => 123
  12  => 37
  75  => 44
  23  => 42
  111 => 66
  ⋮   => ⋮

In [143]:
median_MAX_DAYS_W_OUTGOING = convert(Int64, round(median(skipmissing(df.MAX_DAYS_W_OUTGOING)), digits=0))
df.MAX_DAYS_W_OUTGOING = coalesce.(df.MAX_DAYS_W_OUTGOING, median_MAX_DAYS_W_OUTGOING)

17210-element Vector{Int64}:
  26
 100
 100
 180
 168
 164
   8
 100
   ⋮
 155
 147
 100
  84
 146
 114
   0

In [144]:
median_MAX_DAYS_W_MB = convert(Int64, round(median(skipmissing(df.MAX_DAYS_W_MB)), digits=0))
df.MAX_DAYS_W_MB = coalesce.(df.MAX_DAYS_W_MB, median_MAX_DAYS_W_MB)

17210-element Vector{Int64}:
 182
  81
  81
 169
  82
 128
   8
  81
   ⋮
 111
 156
  74
  79
 173
 151
   0

In [145]:
median_MAX_DAYS_IN_DEBT = convert(Int64, round(median(skipmissing(df.MAX_DAYS_IN_DEBT)), digits=0))
df.MAX_DAYS_IN_DEBT = coalesce.(df.MAX_DAYS_IN_DEBT, median_MAX_DAYS_IN_DEBT)

17210-element Vector{Int64}:
  2
  4
  4
  0
 36
  0
  4
  4
  ⋮
 92
  0
  0
 33
 47
  0
  0

In [146]:
df.MAX_DEBT_SUM = passmissing(replace).(df.MAX_DEBT_SUM, ","=>".")
df.MAX_DEBT_SUM = passmissing(parse).(Float64, df.MAX_DEBT_SUM)
median_MAX_DEBT_SUM = median(skipmissing(df.MAX_DEBT_SUM))
df.MAX_DEBT_SUM = coalesce.(df.MAX_DEBT_SUM, mean_MAX_DEBT_SUM)

17210-element Vector{Float64}:
 14.1
 19.78935953190488
 19.78935953190488
  0.0
 14.99
  0.0
 19.78935953190488
 19.78935953190488
  ⋮
 37.06
  0.0
  0.0
 20.74
  4.64
  0.0
  0.0